In [4]:
import pandas as pd

## Pre-process VMIR output

In [16]:
def parse_vmir_sequence(path):
    with open(path, 'r') as f:
        txt = f.readlines()
        return txt[10].strip()
    
    
def complementary_seq(sequence, reverse=False):
    complement_trans = str.maketrans('ATGC', 'TACG')
    sequence = sequence.translate(complement_trans)
    if reverse:
        sequence = ''.join(reversed(sequence))
    return sequence

    
def get_seq_from_summary(summary, full_sequence):
    seq_start = summary.Start - 1
    sequence = full_seq[seq_start:seq_start+summary.Size]
    if summary.Orientation == 'Reverse':
        sequence = complementary_seq(sequence, reverse=True)
    return sequence

In [6]:
export = pd.read_csv('sequence_no_header_no_whitespace_Export.txt', skiprows=32, sep='\s')
export

<ipython-input-6-3d2f9794c2cd>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  export = pd.read_csv('sequence_no_header_no_whitespace_Export.txt', skiprows=32, sep='\s')


,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.
0,121,MD1,Direct,407,445,72,137.3,0,0,3,3,NaN,NaN
1,85,MD2,Direct,443,490,97,150.6,0,0,1,1,NaN,NaN
2,194,MD3,Direct,596,640,87,120.6,0,0,1,1,NaN,NaN
3,114,MD4,Direct,651,692,75,140.3,0,0,3,3,NaN,NaN
4,95,MD5,Direct,1083,1124,86,147.3,0,0,1,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,203,MR78,Reverse,27942,27986,89,119.5,0,0,2,2,NaN,NaN
218,120,MR79,Reverse,28216,28252,81,137.4,0,0,1,1,NaN,NaN
219,73,MR80,Reverse,28421,28460,80,155.1,0,0,1,1,NaN,NaN
220,101,MR81,Reverse,28925,28966,83,145.5,0,0,1,1,NaN,NaN


In [7]:
full_seq = ''
with open('sequence_no_header_no_whitespace.fasta', 'r') as f:
    full_seq = f.read()
    
len(full_seq)

29903

In [8]:
sequences = []

for index, row in export.iterrows():
    seq = get_seq_from_summary(row, full_seq)
    sequences.append(seq)
    
export['Sequence'] = sequences
export

,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.,Sequence
0,121,MD1,Direct,407,445,72,137.3,0,0,3,3,NaN,NaN,GATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTC...
1,85,MD2,Direct,443,490,97,150.6,0,0,1,1,NaN,NaN,GTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGG...
2,194,MD3,Direct,596,640,87,120.6,0,0,1,1,NaN,NaN,GTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGA...
3,114,MD4,Direct,651,692,75,140.3,0,0,3,3,NaN,NaN,AAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGA...
4,95,MD5,Direct,1083,1124,86,147.3,0,0,1,1,NaN,NaN,TTGTATTTCCCTTAAATTCCATAATCAAGACTATTCAACCAAGGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,203,MR78,Reverse,27942,27986,89,119.5,0,0,2,2,NaN,NaN,TACCATTTAGAATAGAAGTGAATAGGACACGGGTCATCAACTACAT...
218,120,MR79,Reverse,28216,28252,81,137.4,0,0,1,1,NaN,NaN,TTTTGGGGTCCATTATCAGACATTTTAGTTTGTTCGTTTAGATGAA...
219,73,MR80,Reverse,28421,28460,80,155.1,0,0,1,1,NaN,NaN,GTGTTAATTGGAACGCCTTGTCCTCGAGGGAATTTAAGGTCTTCCT...
220,101,MR81,Reverse,28925,28966,83,145.5,0,0,1,1,NaN,NaN,GTTTGGCCTTGTTGTTGTTGGCCTTTACCAGACATTTTGCTCTCAA...


In [10]:
export.to_csv('vmir-pre-mirnas-with-seqs.tsv', sep='\t', index=False)

## Preprocess miRNAFold output

In [28]:
with open('miRNAfold_output.txt', 'r') as f:
    mirnas = f.read().split('\n\n')
    mirnas = [mirna.split('\n')[1].strip() for mirna in mirnas if len(mirna) > 0]
    
    # NOTE: we replace uracil by thymine for compatibility with VMIR output
    mirnas = [mirna.replace('U', 'T') for mirna in mirnas]
    
len(mirnas)

519

# Find common pre-miRNA predictions 

In [56]:
vmir_seqs = set(list(export['Sequence']))
mirnafold_seqs = set(mirnas)

direct_intersection = mirnafold_seqs.intersection(vmir_seqs)
reverse_intersection = set([complementary_seq(seq, reverse=True) for seq in mirnafold_seqs]).intersection(vmir_seqs)
total_intersection = direct_intersection.union(reverse_intersection)

print('Direct intersections:', len(direct_intersection))
print('Reverse intersections:', len(reverse_intersection))

Direct intersections: 2
Reverse intersections: 1


In [53]:
print('miRNA candidates predicted by both VMIR and miRNAFold:')
export[export['Sequence'].isin(total_intersection)]

miRNA candidates predicted by both VMIR and miRNAFold:


,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.,Sequence
4,95,MD5,Direct,1083,1124,86,147.3,0,0,1,1,NaN,NaN,TTGTATTTCCCTTAAATTCCATAATCAAGACTATTCAACCAAGGGT...
39,15,MD40,Direct,7723,7764,89,191.5,0,0,1,1,NaN,NaN,GTCTTCTTACATCGTTGATAGTGTTACAGTGAAGAATGGTTCCATC...
203,187,MR64,Reverse,24722,24762,79,123.0,0,0,1,1,NaN,NaN,CAGGAGCAGTTGTGAAGTTCTTTTCTTGTGCAGGGACATAAGTCAC...


# MFEI calculation

In [1]:
#pre-mirna candidate
#generally lower the better, but at least MFEI ≤ -0.85 kcal/mol per https://www.biorxiv.org/content/10.1101/2020.11.02.365049v1.full.pdf
string = 'GAUUGCUGCAGUCAUAACAAGAGAAGUGGGUUUUGUCGUGCCUGGUUUGCCUGGCACGAUAUUACGCACAACUAAUGGUGACUUUUUGCAUUUC'

def mfei_calculation(precursor, mfe):
    g_and_c = (precursor.count('G')+precursor.count('C'))/len(precursor)
    return mfe/len(precursor)/g_and_c

mfei_calculation(string,-37.3)

-0.8880952380952382